In [11]:
import flask
from werkzeug.serving import run_simple
from werkzeug.wrappers import Request, Response
from random import randint
import pandas as pd
import numpy as np
import datetime
import pickle
from gevent.pywsgi import WSGIServer

from uszipcode import SearchEngine, Zipcode
search = SearchEngine()

app = flask.Flask(__name__)

def read_model_file (model):
    
    with open(model, 'rb') as fp:
        pickled_file = pickle.load(fp)
        print("model is loaded")
    return pickled_file

# 1_Load database data
df2 = pd.read_csv("model_data_29508_rows_2019_+_olddata_rmse_39_v2.csv", index_col=0)

#2_Load pickled model
cost_model=pickle.load(open("model_29508rows_11_cols_simple_model.pkl", 'rb'))

#3_Load pickled model
ht_over_ton_model =pickle.load(open('4_1_model_ht_over_ton_14489rows_9_cols_rmse_11.pkl', 'rb'))

#4_Load pickled model
hwt_over_ton_model =pickle.load(open('4_2_model_hwt_over_ton_14489rows_9_cols_rmse_11.pkl', 'rb'))


# Column name lists @ Cost Model total#11
model_cols=["customer_waste","customer_state",'customer_sub_waste_HWT','customer_sub_waste_HT',
            "date_entered_year","date_entered_month",
            "customer_tons_include_price",'customer_container_size',
            "customer_days_include",
            "short_zip","pop_dens"] #y:"customer_total_price", categorical variable;"customer_sub_waste" ["HT","HWT"]
                                    # wiill be created/predicted in functions ; "ton_diff_x_over_ton","over_ton_price"

# Categorical Variables for dummy_variables @ Haul Model total#4
cat_var1 = ["price_price_deals"]


state_abb_dict = {'AK': 2, 'AL': 1, 'AR': 4, 'AZ': 3, 'CA': 5, 'CO': 6, 'CT': 7, 'DC': 9, 'DE': 8, 'FL': 10, 'GA': 11, 'HI': 12,
 'IA': 16, 'ID': 13, 'IL': 14, 'IN': 15, 'KS': 17, 'KY': 18, 'LA': 19, 'MA': 34, 'MD': 33, 'ME': 20, 'MI': 35, 'MN': 36,
 'MO': 38, 'MS': 37, 'MT': 21, 'NC': 28, 'ND': 29, 'NE': 22, 'NH': 24, 'NJ': 25, 'NM': 26, 'NV': 23, 'NY': 27, 'OH': 30,
 'OK': 31, 'OR': 32, 'PA': 39, 'PR': 53, 'RI': 40, 'SC': 41, 'SD': 42, 'TN': 43, 'TX': 44, 'UT': 45,'VA': 47, 'VI': 54,
 'VT': 46, 'WA': 48, 'WI': 50, 'WV': 49, 'WY': 51, "XX":0}


waste_dict = {'construction':1,"candd":1, 'household':2, 'standard':17, 'roofing':3, 'concrete':5,
       'yard_waste':4, 'dirt':8, 'mixed':7, 'portable handwashing station':10, 'metal':6,
       'flat':11, 'portable restroom trailer':12, 'wheelchair accessible':13, 'dry':14,
       'hazardous':9, 'towable':15, 'special purpose':16}

cols= ['customer_waste', 'customer_state','date_entered_year',
       'date_entered_month', 'customer_tons_include_price',
       'customer_container_size',
       'customer_days_include','short_zip', 'pop_dens',
       'customer_sub_waste_HT','customer_sub_waste_HWT'] # removed from new model :  'ton_diff_x_over_ton', 'over_ton_price',

# Model Columns for Over ton Prediction from file 4_
cols2 = ['customer_container_size', 'customer_tons_include_price',
       'customer_days_include', 'customer_waste', 'customer_state',
       'short_zip', 'date_entered_year', 'date_entered_month']




# This function is used to predict Customer Total Price depending with HT/HWT predicted over ton price ;

def pred_cost(input_dict):
        
    for cat in ["HT", "HWT"]:
        
        # Create local input dictionary
        input_dict1 = input_dict
        
        # Convert input data to dataframe
        input_dict_df1 = pd.DataFrame.from_dict([input_dict1])
        
        input_dict_df1['customer_state'] = input_dict_df1['customer_state'].replace(state_abb_dict)
        input_dict_df1['customer_waste'] = input_dict_df1['customer_waste'].replace(waste_dict)
        
        # Getting over_ton predictions before creating new feature for "customer_tons_include_price"
        over_ton_cost_ht = ht_over_ton_model.predict(input_dict_df1[cols2])
        over_ton_cost_hwt = hwt_over_ton_model.predict(input_dict_df1[cols2])
        
        # create new feature "ACTUAL_customer_tons_include_price" from actual "customer_tons_include_price" to calculate additional tons cost and 
        # sum up to final prediction
        
        input_dict_df1['ACTUAL_customer_tons_include_price']= input_dict_df1['customer_tons_include_price']
        
        # drop "customer_tons_include_price" column. New "customer_tons_include_price" will be created at next step
        input_dict_df1.drop('customer_tons_include_price',axis=1,inplace=True)
       
        # DERIVE new 'customer_tons_include_price' from "customer_container_size" 
          #to use instead of actual"customer_tons_include_price" in prediction model
        
        input_dict_df1['customer_tons_include_price'] = input_dict['customer_container_size']/10 
        
        # get additional tonnage value
        diff = float(input_dict_df1['ACTUAL_customer_tons_include_price']-input_dict_df1['customer_tons_include_price'])
        
        if cat=="HT" :   
                   
            # create dummy_variables for HT/HWT Categories
            input_dict_df1['customer_sub_waste_HT'] = 1
            input_dict_df1['customer_sub_waste_HWT'] = 0              
            
            # Select necessary cols for model
            input_dict_df1 = input_dict_df1[model_cols]
            
            # prediction by pickled object of Haul Price Prediction Model
            cost_pred_ht = cost_model.predict(input_dict_df1[cols])
            
            # caculate new cost with additional tonnage
            
            if diff>0:
                cost_pred_ht = cost_pred_ht+diff*over_ton_cost_ht
            else:
                cost_pred_ht = cost_pred_ht
            
        elif cat=="HWT" : 
  
            # create dummy_variables for HT/HWT Categories
            input_dict_df1['customer_sub_waste_HT'] = 0
            input_dict_df1['customer_sub_waste_HWT'] = 1  

            # Select necessary cols for model
            input_dict_df1 = input_dict_df1[model_cols]
 
            # prediction by pickled object of Haul Price Prediction Model
            cost_pred_hwt = cost_model.predict(input_dict_df1[cols])
            
            # caculate new cost with additional tonnage
            if diff>0:
                cost_pred_hwt = cost_pred_hwt+diff*over_ton_cost_ht
            else:
                cost_pred_hwt = cost_pred_hwt 
                
        
    return cost_pred_ht,cost_pred_hwt,over_ton_cost_ht,over_ton_cost_hwt


# Sum up all functions in one function
def res(input_dict):

    #try:
    pred_HT, pred_HWT, pred_OvTon_HT,pred_OvTon_HWT = pred_cost(input_dict)
    
    res_dict = { 'zip_state': (input_dict['customer_zip'], input_dict['customer_state']),
                "HT": round(int(pred_HT[0])),"HWT": round(int(pred_HWT[0])),
                "Over_TonHT": pred_OvTon_HT,"Over_TonHWT": pred_OvTon_HWT}
    
    return res_dict


# Find State Abbrevation by Zip Function

def zip_to_state_abb(zip_code):
    
    search = SearchEngine()
    NoneType = type(None)

    try:
        zipcode = search.by_zipcode(zip_code)

        count = 0
        while type(zipcode.city)==NoneType:

            zip_code = str(int(zip_code)+1)
            zipcode = search.by_zipcode(zip_code)

            count +=1
            if count > 1000:
                break

        return (zipcode.state_abbr, int(zipcode.population_density)/10)
    except:
        return "XX", 0

# This function is used to crosscheck predicted value with ones in db
def dbcheck(input_dict):
    
    xdf = df2.customer_total_price[(df2.short_zip == int(input_dict['customer_zip'][0:3]))&
                         (df2.customer_container_size == input_dict['customer_container_size'])]
    
    res1=0                                
    
    if xdf.shape[0]>5:
                                   
        res1 = df2.customer_total_price[(df2.short_zip == int(input_dict['customer_zip'][0:3]))&
                             (df2.customer_container_size == input_dict['customer_container_size'])&
                            (df2.customer_tons_include_price== input_dict['customer_tons_include_price'])&
                            (df2.customer_days_include== input_dict['customer_days_include'])].median() 

        if not (float('-inf') < float(res1) < float('inf')):
            res1 = df2.customer_total_price[(df2.short_zip == int(input_dict['customer_zip'][0:3]))&
                             (df2.customer_container_size == input_dict['customer_container_size'])&
                             (df2.customer_tons_include_price== input_dict['customer_tons_include_price'])].median()
        
        if not (float('-inf') < float(res1) < float('inf')):
            res1 = df2.customer_total_price[(df2.short_zip == int(input_dict['customer_zip'][0:3]))&
                             (df2.customer_container_size == input_dict['customer_container_size'])].median()  
    
        if not (float('-inf') < float(res1) < float('inf')):
            res1 = 0
        
        return res1
    
    else:
       
        return res1                          
    

#@app.route('/predict', methods=['POST', 'GET'])
def predict():

    #try:
        #input_dict = flask.request.get_json()

    #except Exception as e:

        #return flask.jsonify({'success': False, "error": str(e)})

    try:
        input_dict['customer_container_size']= float(input_dict['customer_container_size'])
        input_dict['customer_days_include']= float(input_dict['customer_days_include'])
        input_dict['customer_tons_include_price']= float(input_dict['customer_tons_include_price'])
        
    except Exception as e:
        return ({'success': False, "error": str(e)})#flask.jsonify
   
    # Prepare input_dict as modelfeatures

    # Date Conversion/Slicing into units
    date = pd.to_datetime(input_dict["date_entered"])
    input_dict['date_entered_year'] = int(date.year)
    input_dict['date_entered_month'] = int(date.month)

    # Zip Conversion/Get first 3 digits
    input_dict['short_zip'] = float(input_dict['customer_zip'][:3])

    #input_dict['customer_state'] = zip_to_state_abb(int(input_dict['customer_zip']))
    
    input_dict['customer_state'], input_dict['pop_dens'] = zip_to_state_abb(int(input_dict['customer_zip']))

    input_dict["customer_waste"] = input_dict["customer_waste"].lower()
    
    response= res(input_dict)
    
    # Mean Value from DB
    res1 = dbcheck(input_dict)
    
    if  ( (res1!=0) & (float('-inf') < float(res1) < float('inf') ) & (abs(res1-(response["HT"]+response["HWT"])/2)>100)) :
        
        return  ({'success':True, "response":response,"DB":round(res1)})
    else:
        
        return ({'success':True, "response": response})



In [14]:
input_dict = {"date_entered": "2019-05-30 12:00:00",
    'customer_zip':"75149",    
    "customer_container_size": 20.0,
    'customer_days_include': 7.0,
    'customer_tons_include_price': 6,
    'customer_waste': 'candd'}


predict()

{'success': True,
 'response': {'zip_state': ('75149', 'TX'),
  'HT': 888,
  'HWT': 895,
  'Over_TonHT': array([88.0235]),
  'Over_TonHWT': array([75.53])},
 'DB': 572}